In [1]:
%load_ext autoreload
%autoreload 2

# Goal: Run facility choice causal model optimization to find input parameters for Vivarium

The optimization results are output in a `.csv` file at the end of the notebook.

## Requirements:

If you want to run this notebook or any of the code in the
`birth_facility.py` or `solution_finding.py` modules, you will need the right environment.
You can create it with these steps (on 64-bit Linux):

- `conda create --name facility_choice --file environment_lock_conda.txt`
- `conda activate facility_choice`
- `pip install -r environment_lock_pip.txt`

If you're not on 64-bit Linux, replacing the first command with `conda create --name facility_choice python=3.10`
should approximately recreate the environment.

If you want to update all the packages in the environment, create it using `conda env create --name facility_choice --file environment.yaml` (no need for any `pip` commands).
Then, run the following commands inside the environment you created to update the lock files:

- `conda list --explicit > environment_lock_conda.txt`
- `pip freeze | grep -v 'file:///' | grep -v '\-e' > environment_lock_pip.txt`

You will also need to be on the IHME cluster to run this notebook, as the causal model
nanosim loads LBWSG data from an Artifact for the MNCNH Portfolio model.

In [2]:
import pandas as pd

import birth_facility as bf
import solution_finding as sf

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e scipy -e statsmodels -e matplotlib -e gbd

Wed Nov 12 13:40:07 PST 2025
zmbc
Linux long-slurm-sarchive-p0056 5.4.0-216-generic #236-Ubuntu SMP Fri Apr 11 19:53:21 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/homes/zmbc/src/vivarium_gates_mncnh/src/vivarium_gates_mncnh/data/facility_choice
Python 3.10.19
facility_choice       *  /mnt/share/homes/zmbc/mambaforge/envs/facility_choice
gbd                       4.37.1                   pypi_0    pypi
gbd-mapping               5.0.0                    pypi_0    pypi
matplotlib-inline         0.1.7                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    1.5.3                    pypi_0    pypi
pandas-stubs              2.2.3.250308             pypi_0    pypi
scipy                     1.15.3                   pypi_0    pypi
statsmodels               0.14.5                   pypi_0    pypi


# Load data

The input data comes from two sources:

- An artifact for the `vivarium_gates_mncnh` simulation, which is used for population structure, LBWSG exposure and RRs, ANC and IFD coverage, etc.
- The file `facility_choice_data.xlsx` in this folder (downloaded from
  the [master version on
  Sharepoint](https://uwnetid.sharepoint.com/:x:/r/sites/ihme_simulation_science_team/Shared%20Documents/Research/BMGF_MNCH/MNCNH%20portfolio%20products/01_Planning/facility_choice_data.xlsx?d=wf3b3dd5f641f413ba3537c4ca3364cdf&csf=1&web=1&e=cNf9X7)),
  which contains all the other country-specific data to run the model

In [3]:
data_e = bf.BirthFacilityChoiceData('ethiopia')
data_n = bf.BirthFacilityChoiceData('nigeria')
data_p = bf.BirthFacilityChoiceData('pakistan')

# Create models for all 3 countries

Each of the three model objects defines the causal model nanosim that
will be run repeatedly by the `OptimalSolutionFinder.find_solutions`
optimization routine below.

In [4]:
# Choose a population size that's large enough to get stochastically
# stable results, but small enough that the optimization isn't too slow
pop_size = 100_000
# In Vivarium, we will be using the 2-facility-type model (at-home vs.
# in-facility)
num_facility_types = 2

# The random seeds were generated by calling numpy.random.SeedSequence()
# and then copying the generated entropy
model_e = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_e, pop_size, num_facility_types,
    seed=97269740763240770980627172105157383126
)
model_n = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_n, pop_size, num_facility_types,
    seed=230426788122306909803111949239871602900
)
model_p = bf.BirthFacilityModelWithUltrasoundAndSimpleGAError(
    data_p, pop_size, num_facility_types,
    seed=150031875166580400207947577126615739298
)

# Create solution finder objects and specify inputs for optimization

In [5]:
# These are the 5 parameters we need to solve for in the optimization
model_e.parameter_data

attribute,correlated_pair,bounds,position
parameter_name,,,
"corr(anc, lbwsg_category)","(anc, lbwsg_category)","(-1, 1)",0
"corr(anc, facility)","(anc, facility)","(-1, 1)",1
"corr(lbwsg_category, facility)","(lbwsg_category, facility)","(-1, 1)",2
prob_home_given_believed_preterm,<NA>,"(0, 1)",3
prob_home_given_believed_term,<NA>,"(0, 1)",4


In [6]:
finder_e = sf.OptimalSolutionFinder(model_e)
finder_n = sf.OptimalSolutionFinder(model_n)
finder_p = sf.OptimalSolutionFinder(model_p)

# 25 initial conditions seems to be sufficient to get close to the
# global minimum
n_points = 25
# Fix the two LBWSG correlations at 0.2 -- According to the above
# parameter_data dataframe, these parameters are in positions 0 and 2 in
# the parameter vector
fixed_x_components = {0: 0.2, 2: 0.2}

# Find solutions

# Ethiopia

In [7]:
%%time
solutions_e = finder_e.find_solutions(n_points, fixed_x_components)
print(solutions_e.sorted_losses)
solutions_e.sorted_x_values

[1.0134970696018542e-07, 1.0376819403745685e-07, 1.3503852935858873e-07, 3.803948690483949e-07, 8.17985843504232e-07, 9.660440253300706e-07, 1.5345116631815259e-06, 1.6380969828366787e-06, 1.2078494083911373e-05, 5.663339989547822e-05, 8.125676955972327e-05, 0.00014866777753630434, 0.0003354772108552506, 0.000349138433891949, 0.00038702465671736075, 0.00913609462475995, 0.014643862931461915, 0.014925338283898637, 0.02305007221674138, 0.038343238998514884, 0.0565091283182243, 0.07318757471689741, 0.07500124840502131, 0.08862262287942102, inf]
CPU times: user 15min 48s, sys: 3min 36s, total: 19min 25s
Wall time: 13min 57s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.795975,0.2,0.204066,0.619051
1,0.2,0.796024,0.2,0.204600,0.619019
2,0.2,0.795072,0.2,0.205558,0.619192
3,0.2,0.789507,0.2,0.207411,0.615905
4,0.2,0.789454,0.2,0.207416,0.618494
5,0.2,0.800321,0.2,0.205523,0.617537
6,0.2,0.786574,0.2,0.211058,0.615972
7,0.2,0.786637,0.2,0.209965,0.617461
8,0.2,0.818785,0.2,0.190416,0.626555
9,0.2,0.753012,0.2,0.230390,0.599617


In [8]:
data_e.targets_2_facility_types.to_frame().join(solutions_e.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.183369  0.184115  0.187255   
               anc1                     0.664786  0.665154  0.663578   
preterm        at_home                  0.163579  0.163258  0.163058   
               in_facility              0.163486  0.163440  0.164368   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.186903  0.188558  0.184280  0.179688  0.182311   
               anc1         0.662984  0.663775  0.663826  0.665919  0.664138   
preterm        at_home      0.163055  0.163626  0.163768  0.163378  0.163863   
               in_facility  0.164335  0.163723  0.162948  0.163453  0.162854   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.182530  0.184316  ...  0.189978  0.185447   
               anc1         0.662698  0.665770  ...  0.690666  0.586051   
preterm        at_home      0.163757  0.161063  ...  0.244312  0.238249   
               in_facility  0.162937  0.165842  ...  0.105449  0.088230   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.189311  0.227623  0.183208  0.405170  0.517084   
               anc1         0.586708  0.525843  0.412945  0.476490  0.585446   
preterm        at_home      0.239424  0.231522  0.120573  0.228143  0.246403   
               in_facility  0.087435  0.084246  0.238150  0.089189  0.098352   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.515870  0.482328  0.186903  
               anc1         0.563108  0.415480  0.662984  
preterm        at_home      0.244640  0.098017  0.163055  
               in_facility  0.096510  0.247547  0.164335  

[4 rows x 26 columns]

In [9]:
data_e.input_probabilities.to_frame().join(solutions_e.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.515312,0.516510,0.512060,0.512060,0.512060,0.516510,0.516510,0.516510,0.516510,0.515200,...,0.512060,0.512060,0.512060,0.515200,0.516510,0.516510,0.515200,0.513610,0.515200,0.512060
Female,0.484688,0.483490,0.487940,0.487940,0.487940,0.483490,0.483490,0.483490,0.483490,0.484800,...,0.487940,0.487940,0.487940,0.484800,0.483490,0.483490,0.484800,0.486390,0.484800,0.487940
anc0,0.219890,0.219700,0.219700,0.219740,0.219720,0.219720,0.219770,0.219680,0.219690,0.219460,...,0.219710,0.219470,0.219850,0.219310,0.219750,0.219710,0.218630,0.220540,0.218710,0.219740
anc1,0.780110,0.780300,0.780300,0.780260,0.780280,0.780280,0.780230,0.780320,0.780310,0.780540,...,0.780290,0.780530,0.780150,0.780690,0.780250,0.780290,0.781370,0.779460,0.781290,0.780260
preterm,0.163527,0.163360,0.163790,0.163770,0.163680,0.163310,0.163420,0.163300,0.163300,0.163740,...,0.163680,0.163520,0.163530,0.163710,0.163190,0.164110,0.161940,0.162770,0.162330,0.163770
term,0.836473,0.836640,0.836210,0.836230,0.836320,0.836690,0.836580,0.836700,0.836700,0.836260,...,0.836320,0.836480,0.836470,0.836290,0.836810,0.835890,0.838060,0.837230,0.837670,0.836230
at_home,0.441073,0.440530,0.441070,0.441630,0.440640,0.441540,0.440940,0.441710,0.442790,0.439890,...,0.419340,0.501870,0.500660,0.539560,0.637540,0.539180,0.429500,0.447310,0.569900,0.441630
in_facility,0.558927,0.559470,0.558930,0.558370,0.559360,0.558460,0.559060,0.558290,0.557210,0.560110,...,0.580660,0.498130,0.499340,0.460440,0.362460,0.460820,0.570500,0.552690,0.430100,0.558370
BEmONC,0.089922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Nigeria

In [10]:
%%time
solutions_n = finder_n.find_solutions(n_points, fixed_x_components)
print(solutions_n.sorted_losses)
solutions_n.sorted_x_values

[6.907310001746936e-08, 7.604194851573709e-08, 1.191819343215883e-07, 2.8144142705244235e-07, 3.2445888598076067e-07, 2.2130464002456662e-06, 9.073818630800368e-06, 1.266149210843448e-05, 8.484427776211945e-05, 0.0008877851601062359, 0.0013373543162940038, 0.007579921132317979, 0.008287559256255905, 0.008347503150883728, 0.00838768088547992, 0.010863690384482427, 0.017716822208153182, 0.01857619107122721, 0.020095007972543932, 0.020943756240670908, 0.0238593953032602, 0.027055920366467756, 0.032054211808675404, 0.03208094172230769, 0.034530325197076106]
CPU times: user 16min 5s, sys: 3min 27s, total: 19min 32s
Wall time: 15min 27s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.455476,0.2,0.295749,0.630588
1,0.2,0.456916,0.2,0.296378,0.629600
2,0.2,0.455394,0.2,0.294489,0.631064
3,0.2,0.455498,0.2,0.298223,0.628114
4,0.2,0.461694,0.2,0.290267,0.633920
5,0.2,0.462546,0.2,0.289470,0.637978
6,0.2,0.467554,0.2,0.293358,0.625207
7,0.2,0.441702,0.2,0.306193,0.624295
8,0.2,0.450161,0.2,0.329354,0.597635
9,0.2,0.634855,0.2,0.162239,0.705870


In [11]:
data_n.targets_2_facility_types.to_frame().join(solutions_n.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.309069  0.308838  0.309918   
               anc1                     0.584168  0.584147  0.583268   
preterm        at_home                  0.172663  0.172564  0.173261   
               in_facility              0.153000  0.153173  0.152459   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.310665  0.309771  0.310047  0.305845  0.308477   
               anc1         0.582881  0.583177  0.582264  0.582983  0.588243   
preterm        at_home      0.172972  0.174734  0.171779  0.171415  0.173290   
               in_facility  0.152768  0.152225  0.154095  0.154054  0.152742   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.312293  0.308282  ...  0.304490  0.444028   
               anc1         0.581524  0.590554  ...  0.512759  0.473289   
preterm        at_home      0.174199  0.179664  ...  0.228286  0.202243   
               in_facility  0.152008  0.146837  ...  0.085861  0.118057   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.461665  0.441762  0.429242  0.463458  0.491768   
               anc1         0.470721  0.456699  0.681914  0.434784  0.522095   
preterm        at_home      0.175734  0.204979  0.257789  0.182382  0.237808   
               in_facility  0.149658  0.112505  0.103684  0.139814  0.091574   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.529932  0.529872  0.439777  
               anc1         0.589492  0.589554  0.415561  
preterm        at_home      0.248333  0.248327  0.104914  
               in_facility  0.102247  0.102259  0.242100  

[4 rows x 26 columns]

In [12]:
data_n.input_probabilities.to_frame().join(solutions_n.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.512450,0.510960,0.510960,0.510960,0.509000,0.512480,0.510960,0.512480,0.51096,0.51096,...,0.509000,0.512480,0.509000,0.512480,0.510960,0.51154,0.509000,0.511540,0.511540,0.510960
Female,0.487550,0.489040,0.489040,0.489040,0.491000,0.487520,0.489040,0.487520,0.48904,0.48904,...,0.491000,0.487520,0.491000,0.487520,0.489040,0.48846,0.491000,0.488460,0.488460,0.489040
anc0,0.257001,0.257190,0.257100,0.257190,0.257190,0.257090,0.256960,0.256940,0.25697,0.25707,...,0.257020,0.256290,0.256290,0.256360,0.257780,0.25724,0.259350,0.256750,0.256760,0.258040
anc1,0.742999,0.742810,0.742900,0.742810,0.742810,0.742910,0.743040,0.743060,0.74303,0.74293,...,0.742980,0.743710,0.743710,0.743640,0.742220,0.74276,0.740650,0.743250,0.743240,0.741960
preterm,0.162567,0.162610,0.162590,0.162610,0.163190,0.162720,0.162530,0.162680,0.16283,0.16266,...,0.162880,0.163030,0.163520,0.163100,0.162740,0.16356,0.162610,0.164450,0.164450,0.162780
term,0.837433,0.837390,0.837410,0.837390,0.836810,0.837280,0.837470,0.837320,0.83717,0.83734,...,0.837120,0.836970,0.836480,0.836900,0.837260,0.83644,0.837390,0.835550,0.835550,0.837220
at_home,0.486533,0.486660,0.487010,0.487130,0.487140,0.487720,0.488230,0.483640,0.48766,0.48201,...,0.540770,0.534210,0.531600,0.547130,0.383220,0.55784,0.485770,0.425800,0.425770,0.578190
in_facility,0.513467,0.513340,0.512990,0.512870,0.512860,0.512280,0.511770,0.516360,0.51234,0.51799,...,0.459230,0.465790,0.468400,0.452870,0.616780,0.44216,0.514230,0.574200,0.574230,0.421810
BEmONC,0.002271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pakistan

In [13]:
%%time
solutions_p = finder_p.find_solutions(n_points, fixed_x_components)
print(solutions_p.sorted_losses)
solutions_p.sorted_x_values

[1.5951398184554932e-06, 1.7427537666314663e-06, 2.1738488870104433e-06, 2.2791109415276622e-06, 3.1611065000047844e-06, 3.3801490443297055e-06, 3.7936939377702927e-06, 3.783070340013239e-05, 0.0006500378470748469, 0.0011740749587524357, 0.007072475192765659, 0.008232479802116166, 0.009418790114193332, 0.009576308175444748, 0.010354066644160276, 0.010393596637858549, 0.010682926465709675, 0.01071503831076992, 0.010995768954422891, 0.011341233668058837, 0.01581565324866252, 0.020336522008360003, 0.024075843733493252, 0.024696483108561318, inf]
CPU times: user 16min 33s, sys: 3min 52s, total: 20min 26s
Wall time: 14min 24s


parameter_name,"corr(anc, lbwsg_category)","corr(anc, facility)","corr(lbwsg_category, facility)",prob_home_given_believed_preterm,prob_home_given_believed_term
0,0.2,0.361761,0.2,0.129769,0.295613
1,0.2,0.361763,0.2,0.130187,0.295393
2,0.2,0.359948,0.2,0.128282,0.297137
3,0.2,0.367203,0.2,0.129571,0.295846
4,0.2,0.358639,0.2,0.131732,0.294437
5,0.2,0.369479,0.2,0.126615,0.299090
6,0.2,0.363664,0.2,0.128653,0.293144
7,0.2,0.386519,0.2,0.116264,0.299624
8,0.2,0.332544,0.2,0.093389,0.327481
9,0.2,0.374832,0.2,0.174331,0.222152


In [14]:
data_p.targets_2_facility_types.to_frame().join(solutions_p.sorted_targets)

target_probabilities         0         1  \
probability_of given                                                   
in_facility    anc0                     0.556322  0.557514  0.557116   
               anc1                     0.795407  0.795725  0.795650   
preterm        at_home                  0.194529  0.196040  0.196267   
               in_facility              0.155164  0.152890  0.152819   

                                   2         3         4         5         6  \
probability_of given                                                           
in_facility    anc0         0.561299  0.551934  0.561390  0.553803  0.557896   
               anc1         0.795555  0.795094  0.794447  0.795780  0.796727   
preterm        at_home      0.196200  0.194258  0.195223  0.194579  0.194577   
               in_facility  0.152928  0.153964  0.153501  0.153347  0.153962   

                                   7         8  ...        15        16  \
probability_of given                            ...                       
in_facility    anc0         0.551004  0.583367  ...  0.538804  0.521505   
               anc1         0.801660  0.789815  ...  0.763770  0.783738   
preterm        at_home      0.192214  0.177226  ...  0.306918  0.309659   
               in_facility  0.154442  0.159176  ...  0.114571  0.118077   

                                  17        18        19        20        21  \
probability_of given                                                           
in_facility    anc0         0.520836  0.577174  0.564339  0.603055  0.613748   
               anc1         0.779447  0.751641  0.876373  0.671400  0.670017   
preterm        at_home      0.308592  0.302494  0.277993  0.127270  0.287907   
               in_facility  0.117357  0.113142  0.142777  0.184286  0.103175   

                                  22        23        24  
probability_of given                                      
in_facility    anc0         0.635643  0.636916  0.636916  
               anc1         0.653544  0.651451  0.651451  
preterm        at_home      0.281792  0.282482  0.282482  
               in_facility  0.100692  0.099992  0.099992  

[4 rows x 26 columns]

In [15]:
data_p.input_probabilities.to_frame().join(solutions_p.sorted_pop_proportions)

,input_probability,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
subpopulation,,,,,,,,,,,,,,,,,,,,,
Male,0.519811,0.519420,0.519420,0.519420,0.517320,0.515460,0.519420,0.517320,0.519420,0.517320,...,0.517320,0.515460,0.517320,0.519420,0.519420,0.515460,0.51546,0.51867,0.515460,0.515460
Female,0.480189,0.480580,0.480580,0.480580,0.482680,0.484540,0.480580,0.482680,0.480580,0.482680,...,0.482680,0.484540,0.482680,0.480580,0.480580,0.484540,0.48454,0.48133,0.484540,0.484540
anc0,0.075919,0.075460,0.075460,0.075450,0.075480,0.075420,0.075460,0.075480,0.075680,0.075150,...,0.076280,0.076260,0.076550,0.075090,0.075770,0.074620,0.07565,0.07564,0.075740,0.075740
anc1,0.924081,0.924540,0.924540,0.924550,0.924520,0.924580,0.924540,0.924520,0.924320,0.924850,...,0.923720,0.923740,0.923450,0.924910,0.924230,0.925380,0.92435,0.92436,0.924260,0.924260
preterm,0.163933,0.162480,0.162480,0.162540,0.162960,0.162810,0.162520,0.162950,0.162650,0.163250,...,0.163310,0.163340,0.163320,0.162650,0.162690,0.165260,0.16492,0.16368,0.163800,0.163800
term,0.836067,0.837520,0.837520,0.837460,0.837040,0.837190,0.837480,0.837050,0.837350,0.836750,...,0.836690,0.836660,0.836680,0.837350,0.837310,0.834740,0.83508,0.83632,0.836200,0.836200
at_home,0.222744,0.222250,0.222350,0.222120,0.223260,0.223130,0.222480,0.221300,0.217310,0.225700,...,0.253390,0.236260,0.240350,0.261460,0.147270,0.333700,0.33424,0.34781,0.349650,0.349650
in_facility,0.777256,0.777750,0.777650,0.777880,0.776740,0.776870,0.777520,0.778700,0.782690,0.774300,...,0.746610,0.763740,0.759650,0.738540,0.852730,0.666300,0.66576,0.65219,0.650350,0.650350
BEmONC,0.264677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Collect optimization results from the 3 countries

Select the solution with the lowest log loss for each country. The above
solutions dataframes are already sorted by log loss, so we just select
the first row of each.

In [16]:
solutions_by_location = {
    'Ethiopia': solutions_e,
    'Nigeria': solutions_n,
    'Pakistan': solutions_p,
}
best_solutions = pd.concat({
    location: solutions.sorted_x_values.loc[0]
    for location, solutions in solutions_by_location.items()
}, axis=1)
best_solutions

,Ethiopia,Nigeria,Pakistan
parameter_name,,,
"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
"corr(anc, facility)",0.795975,0.455476,0.361761
"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
prob_home_given_believed_preterm,0.204066,0.295749,0.129769
prob_home_given_believed_term,0.619051,0.630588,0.295613


# Save results to file

In [17]:
output_filename = 'facility_choice_optimization_results.csv'
best_solutions.to_csv(output_filename)
pd.read_csv(output_filename)

,parameter_name,Ethiopia,Nigeria,Pakistan
0,"corr(anc, lbwsg_category)",0.200000,0.200000,0.200000
1,"corr(anc, facility)",0.795975,0.455476,0.361761
2,"corr(lbwsg_category, facility)",0.200000,0.200000,0.200000
3,prob_home_given_believed_preterm,0.204066,0.295749,0.129769
4,prob_home_given_believed_term,0.619051,0.630588,0.295613
